In [12]:
from langgraph.graph import START, END, StateGraph
from typing import TypedDict
import subprocess

import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")


class State(TypedDict):
    video_file: str
    audio_file: str
    transcription: str

In [13]:
def extract_audio(state: State):
    output_file = state["video_file"].replace("mp4", "mp3")
    command = [
        "ffmpeg",
        "-i",
        state["video_file"],
        "-filter:a",
        "atempo=2.0",
        output_file,
        "-y",
    ]
    subprocess.run(command)
    return {
        "audio_file": output_file,
    }


def transcribe_audio(state: State):
    client = OpenAI(api_key=api_key)
    with open(state["audio_file"], "rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            response_format="text",
            file=audio_file,
            language="en",
            prompt="",
        )
        return {"transcription": transcription}

In [14]:
graph_builder = StateGraph(State)

graph_builder.add_node("extract_audio", extract_audio)
graph_builder.add_node("transcribe_audio", transcribe_audio)

graph_builder.add_edge(START, "extract_audio")
graph_builder.add_edge("extract_audio", "transcribe_audio")
graph_builder.add_edge("transcribe_audio", END)

graph = graph_builder.compile()


In [15]:
graph.invoke(
    {"video_file": "video.mp4"},
)

{'video_file': 'video.mp4',
 'audio_file': 'video.mp3',
 'transcription': "Running a game requires RAM, which is where the CPU needs to store the game's data for calculation. And of course, graphic assets need to be loaded, so modern games can consume a huge amount of memory. However, there was a time when such sloppy execution wasn't acceptable. There was a masterpiece called Pokemon Red, but the Game Boy it ran on was a paltry device. The CPU performance was so poor that a mistake in coding could cause an explosion. With only 8 kilobits of RAM, uploading data wasn't easy. Creating a single word file today and writing just one sentence would probably exceed 8 kilobits. So, developers at the time had to employ incredible optimization skills when coding. The Pokemon game had a feature called Pokedex, which remembered and counted the Pokemon you already seen. So in the first generation, there were 151 Pokemon, and whether or not you encountered them all had to be recorded in memory. How 